In [1]:
from pathlib import Path

import matplotlib.pyplot as plt


In [2]:
import os, sys
sys.path.append(os.path.abspath('..'))

from src.tagseg.data.acdc_dataset import AcdcDataSet

In [3]:
acdc = AcdcDataSet(filepath='../data/01_raw/acdc-training')

In [5]:
dataset = acdc.load()

In [7]:
acdc._describe()

{'tagged': True, 'only_myo': False}

In [15]:
from kedro.extras.datasets.pickle import PickleDataSet

In [10]:
from kedro.config import ConfigLoader

In [11]:
conf_paths = ["../conf/base", "../conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("parameters*", "parameters*/**")

In [12]:
conf_catalog

{'data_params': {'tagged': True, 'only_myo': False},
 'train_val_split': 0.75,
 'pretrained_model': 'data/06_models/model_cine_tag_v1_sd.pt',
 'batch_size': 32}

In [13]:
# Find out where to store .pt file based on params
acdc_features = acdc._describe()
tagged = acdc_features['tagged']
only_myo = acdc_features['only_myo']

filename: str = 'acdc_dataset'
filename += '_tagged' if tagged else '_cine'
filename += '_only_myo' if only_myo else ''
filename += '.pt'

In [17]:
pds = PickleDataSet(filepath=f'../data/05_model_input/{filename}')

In [18]:
pds.exists()

False

In [12]:
pds.save(dataset)

In [13]:
ds = pds.load()